### Load tensorflow

In [0]:
import numpy as np
np.random.seed(42)

In [0]:
import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(42)

### Read training and test data

In [0]:
import pandas as pd
train_df = pd.read_csv('data/flower_photos/train.csv')
test_df = pd.read_csv('data/flower_photos/test.csv')

In [0]:
#Get class names
class_names = train_df['class'].unique().tolist()
print('Flower classes: ', class_names)

Flower classes:  ['tulips', 'daisy', 'sunflowers', 'dandelion', 'roses']


In [0]:
#Define some parameters
img_size = 224
img_depth = 3  

### Build batch generator

In [0]:
import numpy as np

In [0]:
def batch_generator(df, batchsize=32, train_mode=True):   
    
    img_generator= tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                                   width_shift_range=0.2,
                                                                   height_shift_range=0.2,
                                                                   horizontal_flip=True)
    
    while True:       
        
        
        #Generate random numbers to pick images from dataset
        batch_nums = np.random.randint(0,df.shape[0],batchsize)
        
        #Initialize batch images array
        batch_images = np.zeros((batchsize,img_size, img_size,img_depth))
        
        #Initiate batch label array
        batch_labels = np.zeros((batchsize, len(class_names)))
        
        for i in range(batchsize):
            
            #Load image
            flower_image =  tf.keras.preprocessing.image.load_img('data/' + 
                                                                  df.loc[batch_nums[i]]['image_file'],
                                                                  target_size=(img_size, img_size))
            #Convert to array
            flower_image = tf.keras.preprocessing.image.img_to_array(flower_image)
            
            if(train_mode):
                #Apply transform
                flower_image =  img_generator.random_transform(flower_image)
            
            
            #Get the class
            img_class = df.loc[batch_nums[i]]['class']
            #Conver class to number
            img_class = class_names.index(img_class)
            #Convert class to one hot encoding
            img_class = tf.keras.utils.to_categorical(img_class, num_classes=len(class_names))
            
            #Update batch images and class arrays
            batch_images[i] = flower_image
            batch_labels[i] = img_class
        
        batch_images = tf.keras.applications.resnet50.preprocess_input(batch_images)
        
        yield batch_images, batch_labels          

### Load pre-trained model

In [0]:
model = tf.keras.applications.resnet50.ResNet50(include_top=False, #Do not include FC layer at the end
                                          input_shape=(img_size,img_size, img_depth),
                                          weights='imagenet')

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:
#Set pre-trained model layers to not trainable
for layer in model.layers:
    layer.trainable = False

### Add FC layer for new classes

In [0]:
#get Output layer of Pre0trained model
x = model.output

#Flatten the output to feed to Dense layer
x = tf.keras.layers.Flatten()(x)

#Add one Dense layer
x = tf.keras.layers.Dense(200, activation='relu')(x)

#Add output layer
prediction = tf.keras.layers.Dense(len(class_names),activation='softmax')(x)

### Building final model for Classification

In [0]:
#Using Keras Model class
final_model = tf.keras.models.Model(inputs=model.input, #Pre-trained model input as input layer
                                    outputs=prediction) #Output layer added

In [0]:
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Train the model

In [0]:
#Create train and test generator
batchsize = 64
train_generator = batch_generator(train_df, batchsize=batchsize) #batchsize can be changed
test_generator = batch_generator(test_df, batchsize=batchsize, train_mode=False)

In [0]:
final_model.fit_generator(train_generator, 
                          epochs=50,
                          steps_per_epoch= train_df.shape[0]//batchsize,
                          validation_data=test_generator,
                          validation_steps = test_df.shape[0]//batchsize)

Epoch 1/50
45/45 [==============================] - 61s 1s/step - loss: 4.9838 - acc: 0.6549 - val_loss: 3.7521 - val_acc: 0.7472
Epoch 2/50
45/45 [==============================] - 56s 1s/step - loss: 4.0722 - acc: 0.7319 - val_loss: 3.9182 - val_acc: 0.7457
Epoch 3/50
45/45 [==============================] - 55s 1s/step - loss: 3.5139 - acc: 0.7715 - val_loss: 4.4904 - val_acc: 0.7045
Epoch 4/50
45/45 [==============================] - 55s 1s/step - loss: 3.7259 - acc: 0.7590 - val_loss: 4.4185 - val_acc: 0.7188
Epoch 5/50
45/45 [==============================] - 55s 1s/step - loss: 2.9257 - acc: 0.8087 - val_loss: 3.0131 - val_acc: 0.8040
Epoch 6/50
45/45 [==============================] - 55s 1s/step - loss: 2.9611 - acc: 0.8063 - val_loss: 2.8872 - val_acc: 0.8153
Epoch 7/50
45/45 [==============================] - 56s 1s/step - loss: 2.6134 - acc: 0.8309 - val_loss: 4.3110 - val_acc: 0.7216
Epoch 8/50
45/45 [==============================] - 55s 1s/step - loss: 2.4739 - acc: 0.84

In [0]:
print(len(model.layers))
for layer in model.layers[150:]:
    layer.trainable =  True    

175
